# First Look at US Census Data

**Zipcode:** Socio-economic characteristics and health indicators by US Zipcode

In [1]:
#!pip install census
#!pip install zipcode

In [2]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import zipcode
from census import Census

In [3]:
from config import api_key
c = Census(api_key, year=2015)

In [5]:
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

#Get the Raw Data From Census for all the variables we could want to look at socioeocnomic indicators
zipcode_data = c.acs5.get(("NAME", 
                           "B01003_001E",
                           "B01002_001E",
                           "B19013_001E", 
                           "B19301_001E",
                           "B17001_002E", 
                           "B23025_005E",
                           "B17001A_002E",
                           "B17001B_002E",
                           "B17001I_002E",
                           "B02001_002E",
                           "B02001_003E",
                           "B03001_003E",
                           "B25035_001E",
                           "B25058_001E",
                           "B25064_001E",
                           "B25077_001E",
                           "B15003_002E",
                           "B15003_022E",
                           "B15003_025E",
                           "B15003_024E",
                           "B15003_023E",
                           "B15003_021E",
                           "B15003_018E",
                           "B15003_017E",
                           "B16001_002E",
                           "B16001_003E",
                           "B16001_006E",
                           "B16001_009E"), 
                          {'for': 'zip code tabulation area:*'})

#Snap to a Data Frame
census_1 = pd.DataFrame(zipcode_data)

# Rename the Columns
census_1 = census_1.rename(columns={"B01003_001E": "Population",
                                    "B01002_001E": "Median Age",
                                    "B19013_001E": "Household Income",
                                    "B19301_001E": "Per Capita Income",
                                    "B17001_002E": "Pvty Count",
                                    "B23025_005E": "Unemployed Count",
                                    "B17001A_002E": "Pvty_W",
                                    "B17001B_002E": "Pvty_B",
                                    "B17001I_002E": "Pvty_H",
                                    "B02001_002E": "Pop_W",
                                    "B02001_003E": "Pop_B",
                                    "B03001_003E": "Pop_H",
                                    "B25035_001E": "House_Const_Yr",
                                    "B25058_001E": "Med Ctrct Rent",
                                    "B25064_001E": "Med Gross Rent",
                                    "B25077_001E": "Med Home_Val",
                                    "B15003_002E": "Edu_None",
                                    "B15003_021E": "Edu_Ass",
                                    "B15003_018E": "Edu_GED",
                                    "B15003_017E": "Edu_HS",
                                    "B15003_022E": "Edu_Bach",
                                    "B15003_025E": "Edu_Doct",
                                    "B15003_024E": "Edu_Prof",
                                    "B15003_023E": "Edu_Mast",
                                    "B16001_002E": "Lang_Only_Eng",
                                    "B16001_003E": "Lang_Sp",
                                    "B16001_006E": "Lang_Fr",
                                    "B16001_009E": "Lang_Fr|Cr",
                                    "NAME": "Name", 
                                    "zip code tabulation area": "Zipcode"})

# Now we've got all our socio economic variables in one dataframe
census_1.head()

,Median Age,Population,Pop_W,Pop_B,Pop_H,Edu_None,Edu_HS,Edu_GED,Edu_Ass,Edu_Bach,...,Pvty Count,Household Income,Per Capita Income,Unemployed Count,House_Const_Yr,Med Ctrct Rent,Med Gross Rent,Med Home_Val,Name,Zipcode
0,37.6,17982.0,16956.0,102.0,17942.0,456.0,3017.0,156.0,615.0,1720.0,...,10861.0,10816.0,7453.0,2213.0,1982.0,276.0,359.0,104500.0,ZCTA5 00601,00601
1,39.5,40260.0,23144.0,693.0,37681.0,955.0,4770.0,1004.0,3499.0,4203.0,...,21720.0,16079.0,8474.0,3807.0,1980.0,318.0,395.0,90200.0,ZCTA5 00602,00602
2,39.9,52408.0,36177.0,2057.0,50437.0,1014.0,8309.0,1381.0,3147.0,5981.0,...,25459.0,16804.0,10179.0,3991.0,1977.0,288.0,384.0,128400.0,ZCTA5 00603,00603
3,40.8,6331.0,4399.0,151.0,6323.0,188.0,1441.0,74.0,155.0,385.0,...,4070.0,12512.0,5863.0,179.0,1978.0,189.0,280.0,108500.0,ZCTA5 00606,00606
4,40.2,28328.0,17791.0,950.0,27999.0,377.0,5707.0,803.0,1598.0,2266.0,...,14005.0,17475.0,8452.0,1495.0,1979.0,333.0,424.0,113200.0,ZCTA5 00610,00610


In [48]:
# Split the data frame into smaller frames to work on specific factors, make sure that ZIPCODE is included
# in each dataframe

# Population
Population_df = census_1[["Zipcode", "Population", "Pop_W", "Pop_B", "Pop_H", "Median Age"]]
Population_df = Population_df.copy()
Population_df.head()

#Poverty
Poverty_df = census_1[["Zipcode", "Population", "Household Income", "Per Capita Income", "Pvty Count", "Unemployed Count",
                      "Pvty_W", "Pvty_B", "Pvty_H"]]
Poverty_df = Poverty_df.copy()
Poverty_df["Pvty Rate"] = 100 * Poverty_df["Pvty Count"].astype(int) / Poverty_df["Population"].astype(int)
Poverty_df["Unemployed Rate"] = 100 * Poverty_df["Unemployed Count"].astype(int) / Poverty_df["Population"].astype(int)
# Please note that this inflates the unemployed rate because Population includes children, whereas unemployed count 
# is only for people aged 16+
Poverty_df.head()

#Housing
Housing_df = census_1[["Zipcode", "House_Const_Yr", "Med Ctrct Rent", "Med Gross Rent", "Med Home_Val"]]
Housing_df = Housing_df.copy()
Housing_df.head()

#Education
Edu_df = census_1[["Zipcode", "Population", "Edu_None", "Edu_Ass", "Edu_GED", "Edu_HS", "Edu_Bach", 
                   "Edu_Mast", "Edu_Prof", "Edu_Doct"]]
Edu_df = Edu_df.copy()
Edu_df.head()

#Language
Lang_df = census_1[["Zipcode", "Population", "Lang_Only_Eng", "Lang_Sp", "Lang_Fr", "Lang_Fr|Cr"]]
Lang_df = Lang_df.copy()
Lang_df.head()

,Zipcode,Population,Lang_Only_Eng,Lang_Sp,Lang_Fr,Lang_Fr|Cr
0,00601,17982.0,1164.0,15812.0,0.0,0.0
1,00602,40260.0,1437.0,36761.0,0.0,0.0
2,00603,52408.0,2485.0,47098.0,0.0,0.0
3,00606,6331.0,213.0,5771.0,0.0,0.0
4,00610,28328.0,700.0,26190.0,0.0,0.0


In [ ]:
# DON'T NEED TO RE-DO

Zipcode_Geo = pd.read_csv("2015_Gaz_zcta_national_pruned.csv")
Zip_Geo_df = pd.DataFrame(Zipcode_Geo)
print(len(Zip_Geo_df))
Zip_Geo_df.head()
## Need to create full zipcodes

In [ ]:
# DON'T NEED TO RE-DO 

## Correct the zipcodes so that they are all five digits

full_zips = [zip for zip in Zip_Geo_df["Zipcode"]]
correct_zips = []

for zip in full_zips:
    while len(str(zip)) < 5:
        zip = "0" + str(zip)
    correct_zips.append(zip)
        

correct_zips

In [ ]:
# DON'T NEED TO RE-DO

## Get the state for every zipcode using the zipcode plug-in
zc_states = []

for x in range(len(correct_zips)):
    val=zipcode.isequal(str(correct_zips[x]))
    zc_states.append(val.state)

print(len(zc_states))

In [ ]:
# DON'T NEED TO RE-DO

# Make some adjustments to clean the Geo Data dataframe
Zip_Geo_df["State"] = zc_states
Zip_Geo_df["Full Zip"] = correct_zips
Zip_Geo_df.head()

Zip_Geo_klaar = Zip_Geo_df[["Full Zip", "State", "Land Area (sq mi)", "Center Point Lat", "Center Point Lng"]]
Zip_Geo_klaar = Zip_Geo_klaar.rename(columns={"Full Zip": "Zipcode"})
Zip_Geo_klaar.head()

In [ ]:
# Save the Zip Geo correct dataframe to a csv so that we don't have to re-do the state-finder code
# which took 6 minutes to run
Zip_Geo_klaar.to_csv("Zipcode_Geo_Data_Ready.csv", index=False)

# Strange that when I open the file in Excel, you don't see the leading 0's in certain zipcodes

In [8]:
# START BACK HERE

# However, when I read the csv back into a pandas df, as long as I use the 'converters' code, the leading 0's are back

Zip_2 = pd.read_csv("Zipcode_Geo_Data_ready.csv", converters={'Zipcode': lambda x: str(x)})
Zip_2.head()

,Zipcode,State,Land Area (sq mi),Center Point Lat,Center Point Lng
0,00601,PR,64.348,18.180555,-66.749961
1,00602,PR,30.613,18.361945,-67.175597
2,00603,PR,31.616,18.455183,-67.119887
3,00606,PR,42.309,18.158345,-66.932911
4,00610,PR,35.916,18.295366,-67.125135


In [9]:
#Merge population_df with the Geo df so that we can get the Pop Density
Basic_Pop = pd.merge(Population_df, Zip_2, on="Zipcode", how="left")
Basic_Pop.head()

,Zipcode,Population,Pop_W,Pop_B,Pop_H,Median Age,State,Land Area (sq mi),Center Point Lat,Center Point Lng
0,00601,17982.0,16956.0,102.0,17942.0,37.6,PR,64.348,18.180555,-66.749961
1,00602,40260.0,23144.0,693.0,37681.0,39.5,PR,30.613,18.361945,-67.175597
2,00603,52408.0,36177.0,2057.0,50437.0,39.9,PR,31.616,18.455183,-67.119887
3,00606,6331.0,4399.0,151.0,6323.0,40.8,PR,42.309,18.158345,-66.932911
4,00610,28328.0,17791.0,950.0,27999.0,40.2,PR,35.916,18.295366,-67.125135


In [10]:
PDensity = Basic_Pop["Population"]/Basic_Pop["Land Area (sq mi)"]
Basic_Pop["Pop Density"] = PDensity
Basic_Pop.head()

,Zipcode,Population,Pop_W,Pop_B,Pop_H,Median Age,State,Land Area (sq mi),Center Point Lat,Center Point Lng,Pop Density
0,00601,17982.0,16956.0,102.0,17942.0,37.6,PR,64.348,18.180555,-66.749961,279.449245
1,00602,40260.0,23144.0,693.0,37681.0,39.5,PR,30.613,18.361945,-67.175597,1315.127560
2,00603,52408.0,36177.0,2057.0,50437.0,39.9,PR,31.616,18.455183,-67.119887,1657.641700
3,00606,6331.0,4399.0,151.0,6323.0,40.8,PR,42.309,18.158345,-66.932911,149.637193
4,00610,28328.0,17791.0,950.0,27999.0,40.2,PR,35.916,18.295366,-67.125135,788.729257


In [ ]:
# THIS IS JUST A CHECK

# Playing around to see if we should cut off some of the zipcodes because they are strange (i.e. zipcodes w/ 0 ppl)
# Tried this first with Population total, but the lower bound is way below 0, and 3,010 zip codes are outside the 
# upper bound.

# Also tried with Population Density, results even more skewed. Lower Bound is way below 0, 5,633 zipcodes are outside
# the upper bound. Long and the short of it is that we have SKEWED data

Mean = np.mean(Basic_Pop["Pop Density"])
LwQt = np.percentile(Basic_Pop["Pop Density"],25)
UpQt = np.percentile(Basic_Pop["Pop Density"],75)
IQR = UpQt - LwQt
print(f'Upper Quartile: {UpQt}, Lower Quartile: {LwQt}, IQR: {IQR}')
Up_Bound = UpQt + (1.5*IQR)
Low_Bound = LwQt - (1.5*IQR)
print(f'Upper Bound: {Up_Bound}, Lower Bound: {Low_Bound}')
print(len(Basic_Pop.loc[Basic_Pop["Pop Density"] > Up_Bound]))

In [ ]:
#Basic_Pop.to_csv("Pop_w_Density.csv", index=False)
Edu_df.head()
Edu_df.to_csv("edu_data.csv")

In [11]:
#Education
## Remember that Education variables only count population 25 and above, so shouldn't compare vs pop figures.
Full_edu_able_pop = Edu_df["Edu_None"] + Edu_df["Edu_HS"] + Edu_df["Edu_GED"] + Edu_df["Edu_Ass"] + Edu_df["Edu_Bach"] \
                    + Edu_df["Edu_Mast"] + Edu_df["Edu_Prof"] + Edu_df["Edu_Doct"]

One_pt = (Edu_df["Edu_HS"] + Edu_df["Edu_GED"]) / Full_edu_able_pop * 100
Two_pt = (Edu_df["Edu_Ass"] + Edu_df["Edu_Bach"]) / Full_edu_able_pop * 100
Three_pt = (Edu_df["Edu_Mast"] + Edu_df["Edu_Prof"] + Edu_df["Edu_Doct"]) / Full_edu_able_pop * 100
Zero_pt = Edu_df["Edu_None"] / Full_edu_able_pop * 100
Edu_df_2 = pd.DataFrame({"Zipcode": Edu_df["Zipcode"],
                         "Zero_Pt":Zero_pt,
                         "One_Pt": One_pt,
                         "Two_Pt": Two_pt,
                         "Three_Pt": Three_pt})

Education_score = (Edu_df_2["One_Pt"] + Edu_df_2["Two_Pt"]*2 + Edu_df_2["Three_Pt"]*3) / 3
Edu_scores = pd.DataFrame({
    "Zipcode": Edu_df_2["Zipcode"],
    "Population": Edu_df["Population"],
    "Edu Score": Education_score
})
Edu_scores.head()

,Edu Score,Population,Zipcode
0,47.132786,17982.0,00601
1,54.318233,40260.0,00602
2,52.424037,52408.0,00603
3,40.423692,6331.0,00606
4,47.259793,28328.0,00610


In [12]:
Edu_scores["Edu Score"] = Edu_scores["Edu Score"].fillna(0)
Edu_scores.to_csv("Education_check.csv", index=False)

# The education scores range from 0 (no one has any education) to 100 (everyone has a HS degree) to 300 (everyone has
# a professional degree)
# I decided to normalize these scores by dividing the whole thing by 3

In [13]:
# Open the csv describing the municipality type of each zipcode (Rural/Urban/Mixed)

Rur_urb_df = pd.read_csv("Urb_type_ready.csv")
Rur_urb_df = Rur_urb_df[["ZCTA5", "Final Type"]]
Rur_urb_df = Rur_urb_df.rename(columns={"ZCTA5": "Zipcode"})

# Try setting the zipcodes to "to_numeric" in both datasets before trying to combine.
Rur_urb_df["Zipcode"] = Rur_urb_df["Zipcode"].astype(int)
Basic_Pop["Zipcode"] = Basic_Pop["Zipcode"].astype(int)

# Merge with basic pop data
Full_pop_stats = pd.merge(Basic_Pop, Rur_urb_df, on="Zipcode", how="left")
Full_pop_stats.to_csv("Full_pop.csv", index=False)


In [ ]:
john = Full_pop_stats.loc[Full_pop_stats["Zipcode"]==6511,:]
john

In [ ]:
# Correct the zipcodes to all have five digits and be formatted as strings
Full_pop_stats["Zipcode"] = Full_pop_stats["Zipcode"].astype(str)
full_zips_2 = [zip for zip in Full_pop_stats["Zipcode"]]
correct_zips_2 = []

for zip in full_zips_2:
    while len(str(zip)) < 5:
        zip = "0" + str(zip)
    correct_zips_2.append(zip)
    
Full_pop_stats["Zipcode"] = correct_zips_2
Full_pop_stats.iloc[5000:5050,:]

In [16]:
# Now let's get the columns we want to use in our final analysis, starting with those in Full_pop_stats
Full_pop_stats["Ppct_W"] = Full_pop_stats["Pop_W"] / Full_pop_stats["Population"] * 100
Full_pop_stats["Ppct_B"] = Full_pop_stats["Pop_B"] / Full_pop_stats["Population"] * 100
Full_pop_stats["Ppct_H"] = Full_pop_stats["Pop_H"] / Full_pop_stats["Population"] * 100
Full_pop_stats["Ppct_O"] = 100 - (Full_pop_stats["Ppct_W"] + Full_pop_stats["Ppct_B"])
Full_pop_stats.head()

,Zipcode,Population,Pop_W,Pop_B,Pop_H,Median Age,State,Land Area (sq mi),Center Point Lat,Center Point Lng,Pop Density,Final Type,Ppct_W,Ppct_B,Ppct_H,Ppct_O
0,00601,17982.0,16956.0,102.0,17942.0,37.6,PR,64.348,18.180555,-66.749961,279.449245,Mixed,94.294294,0.567234,99.777555,5.138472
1,00602,40260.0,23144.0,693.0,37681.0,39.5,PR,30.613,18.361945,-67.175597,1315.127560,Urbanized Area,57.486339,1.721311,93.594138,40.792350
2,00603,52408.0,36177.0,2057.0,50437.0,39.9,PR,31.616,18.455183,-67.119887,1657.641700,Urbanized Area,69.029537,3.924973,96.239124,27.045489
3,00606,6331.0,4399.0,151.0,6323.0,40.8,PR,42.309,18.158345,-66.932911,149.637193,Mixed,69.483494,2.385089,99.873638,28.131417
4,00610,28328.0,17791.0,950.0,27999.0,40.2,PR,35.916,18.295366,-67.125135,788.729257,Urbanized Area,62.803587,3.353572,98.838605,33.842841


In [18]:
## THIS FIELD IS FOR TESTS

#Testing = Full_pop_stats
#Testing = Full_pop_stats.loc[Full_pop_stats["State"] != "PR",:]
#Testing = Testing.sort_values("Ppct_B", ascending=False)
#Testing.head()
Full_pop_stats.iloc[0,15]

5.1384718051384795

In [ ]:
# Create an indicator for neighborhood ethincity dominance. I don't think we should ONLY have this, 
# but it is interesting. We will have a column with neighborhood dominance (W,B,H,O), then a second stating the percentage
# If a neighborhood doesn't have 75% of one ethnicity, we will call it (Mixed), and put the % of white people.
Neighborhood_eth = []
TEP = []
for x in range(len(Full_pop_stats)):
    if (Full_pop_stats.iloc[x,14] >= 75):
        Neighborhood_eth.append("His")
        TEP.append(Full_pop_stats.iloc[x,14])
    elif (Full_pop_stats.iloc[x,12] >=75):
        Neighborhood_eth.append("White")
        TEP.append(Full_pop_stats.iloc[x,12])
    elif (Full_pop_stats.iloc[x,13] >= 75):
        Neighborhood_eth.append("Black")
        TEP.append(Full_pop_stats.iloc[x,13])
    elif (Full_pop_stats.iloc[x,15] >= 75):
        Neighborhood_eth.append("Other")
        TEP.append(Full_pop_stats.iloc[x,15])
    else:
        Neighborhood_eth.append("Mixed")
        TEP.append(Full_pop_stats.iloc[x,12])

In [ ]:
Full_pop_stats["Neighborhood Eth"] = Neighborhood_eth
Full_pop_stats["Pct Dom Eth"] = TEP
Full_pop_stats.head()
# Start to sort out the columns we don't need
Final_SE_df = Full_pop_stats[["Zipcode", "State", "Final Type", "Center Point Lat", "Center Point Lng", 
                             "Population", "Pop Density", "Median Age", "Neighborhood Eth", "Pct Dom Eth"]]
Final_SE_df = Final_SE_df.rename(columns={"Final Type": "Urban Type", "Neighborhood Eth":"ZC Eth"})
Final_SE_df.head()

In [35]:
print(len(TEP))

33120


In [25]:
Final_SE_df = pd.merge(Final_SE_df, Edu_scores, on="Zipcode", how="left")
Final_SE_df.head()

,Zipcode,State,Urban Type,Center Point Lat,Center Point Lng,Population_x,Pop Density,Median Age,ZC Eth,Pct Dom Eth,Edu Score,Population_y
0,00601,PR,Mixed,18.180555,-66.749961,17982.0,279.449245,37.6,His,99.777555,47.132786,17982.0
1,00602,PR,Urbanized Area,18.361945,-67.175597,40260.0,1315.127560,39.5,His,93.594138,54.318233,40260.0
2,00603,PR,Urbanized Area,18.455183,-67.119887,52408.0,1657.641700,39.9,His,96.239124,52.424037,52408.0
3,00606,PR,Mixed,18.158345,-66.932911,6331.0,149.637193,40.8,His,99.873638,40.423692,6331.0
4,00610,PR,Urbanized Area,18.295366,-67.125135,28328.0,788.729257,40.2,His,98.838605,47.259793,28328.0


In [40]:
#del Final_SE_df["Population_y"]
Final_SE_df = Final_SE_df.rename(columns={"Population_x": "Population"})

17982.0

In [45]:
Final_SE_df.head()

,Zipcode,State,Urban Type,Center Point Lat,Center Point Lng,Population,Pop Density,Median Age,ZC Eth,Pct Dom Eth,Edu Score
0,00601,PR,Mixed,18.180555,-66.749961,17982.0,279.449245,37.6,His,99.777555,47.132786
1,00602,PR,Urbanized Area,18.361945,-67.175597,40260.0,1315.127560,39.5,His,93.594138,54.318233
2,00603,PR,Urbanized Area,18.455183,-67.119887,52408.0,1657.641700,39.9,His,96.239124,52.424037
3,00606,PR,Mixed,18.158345,-66.932911,6331.0,149.637193,40.8,His,99.873638,40.423692
4,00610,PR,Urbanized Area,18.295366,-67.125135,28328.0,788.729257,40.2,His,98.838605,47.259793


In [44]:
Final_SE_df.to_csv("Final_SE_df.csv", index=False)

In [58]:
# Work on Language
#Lang_df["French"] = Lang_df["Lang_Fr"] + Lang_df["Lang_Fr|Cr"]
#Lang_df = Lang_df[["Zipcode", "Population", "Lang_Only_Eng", "Lang_Sp", "French"]]

Lang_df["Eng_Pct"] = Lang_df["Lang_Only_Eng"] / Lang_df["Population"] * 100
Lang_df["Sp_Pct"] = Lang_df["Lang_Sp"] / Lang_df["Population"] * 100
Lang_df["Fr_Pct"] = Lang_df["French"] / Lang_df["Population"] * 100
Lang_df["Oth_Pct"] = 100 - (Lang_df["Eng_Pct"] + Lang_df["Sp_Pct"] + Lang_df["Fr_Pct"])

Lang_df.head()

,Zipcode,Population,Lang_Only_Eng,Lang_Sp,French,91.67019027484143,Eng_Pct,Sp_Pct,Fr_Pct,Oth_Pct
0,00601,17982.0,1164.0,15812.0,0.0,91.67019,6.473140,87.932377,0.0,5.594483
1,00602,40260.0,1437.0,36761.0,0.0,91.67019,3.569300,91.308992,0.0,5.121709
2,00603,52408.0,2485.0,47098.0,0.0,91.67019,4.741642,89.867959,0.0,5.390398
3,00606,6331.0,213.0,5771.0,0.0,91.67019,3.364397,91.154636,0.0,5.480967
4,00610,28328.0,700.0,26190.0,0.0,91.67019,2.471053,92.452697,0.0,5.076250


In [61]:
# Create a dominant language and percentage category
Main_Lang = []
Lng_Pct = []
for x in range(len(Lang_df)):
    if (Lang_df.iloc[x,5] >= 75):
        Main_Lang.append("Eng")
        Lng_Pct.append(Lang_df.iloc[x,5])
    elif (Lang_df.iloc[x,6] >=75):
        Main_Lang.append("Sp")
        Lng_Pct.append(Lang_df.iloc[x,6])
    elif (Lang_df.iloc[x,7] >= 75):
        Main_Lang.append("Fr")
        Lng_Pct.append(Lang_df.iloc[x,7])
    elif (Lang_df.iloc[x,8] >= 75):
        Main_Lang.append("Oth")
        Lng_Pct.append(Lang_df.iloc[x,8])
    else:
        Main_Lang.append("Mixed")
        Lng_Pct.append(Lang_df.iloc[x,5])
        
Lang_df["Main_Lang"] = Main_Lang
Lang_df["Lng_Pct"] = Lng_Pct
Lang_df.head()

,Zipcode,Population,Lang_Only_Eng,Lang_Sp,French,Eng_Pct,Sp_Pct,Fr_Pct,Oth_Pct,Main_Lang,Lng_Pct
0,00601,17982.0,1164.0,15812.0,0.0,6.473140,87.932377,0.0,5.594483,Sp,87.932377
1,00602,40260.0,1437.0,36761.0,0.0,3.569300,91.308992,0.0,5.121709,Sp,91.308992
2,00603,52408.0,2485.0,47098.0,0.0,4.741642,89.867959,0.0,5.390398,Sp,89.867959
3,00606,6331.0,213.0,5771.0,0.0,3.364397,91.154636,0.0,5.480967,Sp,91.154636
4,00610,28328.0,700.0,26190.0,0.0,2.471053,92.452697,0.0,5.076250,Sp,92.452697


In [63]:
Lang_df_ToMerge = Lang_df[["Zipcode", "Main_Lang","Lng_Pct"]]
Lang_df_ToMerge.head()

,Zipcode,Main_Lang,Lng_Pct
0,00601,Sp,87.932377
1,00602,Sp,91.308992
2,00603,Sp,89.867959
3,00606,Sp,91.154636
4,00610,Sp,92.452697


In [64]:
# Merge in the language variables with Final_SE_df
Final_SE_df = pd.merge(Final_SE_df, Lang_df_ToMerge, on="Zipcode", how="left")
Final_SE_df.head()

,Zipcode,State,Urban Type,Center Point Lat,Center Point Lng,Population,Pop Density,Median Age,ZC Eth,Pct Dom Eth,Edu Score,Main_Lang,Lng_Pct
0,00601,PR,Mixed,18.180555,-66.749961,17982.0,279.449245,37.6,His,99.777555,47.132786,Sp,87.932377
1,00602,PR,Urbanized Area,18.361945,-67.175597,40260.0,1315.127560,39.5,His,93.594138,54.318233,Sp,91.308992
2,00603,PR,Urbanized Area,18.455183,-67.119887,52408.0,1657.641700,39.9,His,96.239124,52.424037,Sp,89.867959
3,00606,PR,Mixed,18.158345,-66.932911,6331.0,149.637193,40.8,His,99.873638,40.423692,Sp,91.154636
4,00610,PR,Urbanized Area,18.295366,-67.125135,28328.0,788.729257,40.2,His,98.838605,47.259793,Sp,92.452697


In [70]:
# Housing: All I will do is parse out Med Ctrct Rent (b/c it will correlate perfectly with Med Gross Rent) and merge
# with Final_SE_df
Housing_df_ToMerge = Housing_df[["Zipcode", "House_Const_Yr", "Med Gross Rent", "Med Home_Val"]]
Final_SE_df = pd.merge(Final_SE_df, Housing_df_ToMerge, on="Zipcode", how="left")
Final_SE_df.head()

,Zipcode,State,Urban Type,Center Point Lat,Center Point Lng,Population,Pop Density,Median Age,ZC Eth,Pct Dom Eth,Edu Score,Main_Lang,Lng_Pct,House_Const_Yr,Med Gross Rent,Med Home_Val
0,00601,PR,Mixed,18.180555,-66.749961,17982.0,279.449245,37.6,His,99.777555,47.132786,Sp,87.932377,1982.0,359.0,104500.0
1,00602,PR,Urbanized Area,18.361945,-67.175597,40260.0,1315.127560,39.5,His,93.594138,54.318233,Sp,91.308992,1980.0,395.0,90200.0
2,00603,PR,Urbanized Area,18.455183,-67.119887,52408.0,1657.641700,39.9,His,96.239124,52.424037,Sp,89.867959,1977.0,384.0,128400.0
3,00606,PR,Mixed,18.158345,-66.932911,6331.0,149.637193,40.8,His,99.873638,40.423692,Sp,91.154636,1978.0,280.0,108500.0
4,00610,PR,Urbanized Area,18.295366,-67.125135,28328.0,788.729257,40.2,His,98.838605,47.259793,Sp,92.452697,1979.0,424.0,113200.0


In [74]:
Final_SE_df["Main_Lang"].value_counts()

Eng      27291
Mixed     5480
Sp         298
Oth         51
Name: Main_Lang, dtype: int64

In [ ]:
# Poverty
Poverty_df.head()
# Create columns to describe ethnic poverty in the same style as those for language and dominant ethnicity
# Main group in poverty (W,B,H,O), and % of poverty count from that group. 
# (We could also bring back the population count for each of those groups to see what % of that group is in poverty,
# but I think this will be enough for now)

Poverty_df["Pct_W"] = Poverty_df["Pvty_W"] / Poverty_df["Pvty Count"] * 100
Poverty_df["Pct_B"] = Poverty_df["Pvty_B"] / Poverty_df["Pvty Count"] * 100
Poverty_df["Pct_H"] = Poverty_df["Pvty_H"] / Poverty_df["Pvty Count"] * 100
Poverty_df["Pct_O"] = 100 - (Poverty_df["Pct_W"] + Poverty_df["Pct_B"])
Poverty_df.head()

In [96]:
Poverty_df.head()

,Zipcode,Population,Household Income,Per Capita Income,Pvty Count,Unemployed Count,Pvty_W,Pvty_B,Pvty_H,Pvty Rate,Unemployed Rate,Pct_W,Pct_B,Pct_H,Pct_O
0,00601,17982.0,10816.0,7453.0,10861.0,2213.0,10088.0,71.0,10821.0,60.399288,12.306751,92.882792,0.653715,99.631710,6.463493
1,00602,40260.0,16079.0,8474.0,21720.0,3807.0,13055.0,326.0,20349.0,53.949329,9.456036,60.105893,1.500921,93.687845,38.393186
2,00603,52408.0,16804.0,10179.0,25459.0,3991.0,17349.0,846.0,24701.0,48.578461,7.615250,68.144860,3.322990,97.022664,28.532150
3,00606,6331.0,12512.0,5863.0,4070.0,179.0,2539.0,106.0,4062.0,64.286843,2.827357,62.383292,2.604423,99.803440,35.012285
4,00610,28328.0,17475.0,8452.0,14005.0,1495.0,8529.0,661.0,13840.0,49.438718,5.277464,60.899679,4.719743,98.821849,34.380578


In [115]:
Pov_Grp = []
Pov_Pct = []
for x in range(len(Poverty_df)):
    if (Poverty_df.iloc[x,1] == 0):
        Pov_Grp.append("No_Pop")
        Pov_Pct.append(0)
    elif (Poverty_df.iloc[x,4] == 0):
        Pov_Grp.append("No_Pvty")
        Pov_Pct.append(0)
    elif (Poverty_df.iloc[x,13] >= Poverty_df.iloc[x,11]) and (Poverty_df.iloc[x,13] >= Poverty_df.iloc[x,12]) and (Poverty_df.iloc[x,13] >= Poverty_df.iloc[x,14]):
        Pov_Grp.append("His")
        Pov_Pct.append(Poverty_df.iloc[x,13])
    elif (Poverty_df.iloc[x,12] > Poverty_df.iloc[x,11]) and (Poverty_df.iloc[x,12] > Poverty_df.iloc[x,14]):
        Pov_Grp.append("Black")
        Pov_Pct.append(Poverty_df.iloc[x,12])
    elif (Poverty_df.iloc[x,11] > Poverty_df.iloc[x,14]) and (Poverty_df.iloc[x,11] > Poverty_df.iloc[x,12]):
        Pov_Grp.append("White")
        Pov_Pct.append(Poverty_df.iloc[x,11])
    elif (Poverty_df.iloc[x,14] > Poverty_df.iloc[x,11]) and (Poverty_df.iloc[x,14] > Poverty_df.iloc[x,12]):
        Pov_Grp.append("Other")
        Pov_Pct.append(Poverty_df.iloc[x,14])
    elif (Poverty_df.iloc[x,12] == Poverty_df.iloc[x,11]) and (Poverty_df.iloc[x,12] > Poverty_df.iloc[x,13]) and (Poverty_df.iloc[x,12] > Poverty_df.iloc[x,14]):
        Pov_Grp.append("Mixed B-W")
        Pov_Pct.append(Poverty_df.iloc[x,12])
    elif (Poverty_df.iloc[x,12] == Poverty_df.iloc[x,14]) and (Poverty_df.iloc[x,12] > Poverty_df.iloc[x,13]) and (Poverty_df.iloc[x,12] > Poverty_df.iloc[x,11]):
        Pov_Grp.append("Mixed B-O")
        Pov_Pct.append(Poverty_df.iloc[x,12])
    elif (Poverty_df.iloc[x,11] == Poverty_df.iloc[x,14]) and (Poverty_df.iloc[x,11] > Poverty_df.iloc[x,13]) and (Poverty_df.iloc[x,11] > Poverty_df.iloc[x,12]):
        Pov_Grp.append("Mixed W-O")
        Pov_Pct.append(Poverty_df.iloc[x,11])
    else:
        Pov_Grp.append("None")
        Pov_Pct.append(0)
        
#Poverty_df["Pov_Group"] = Pov_Grp
#Poverty_df["Pov_Pct"] = Pov_Pct
#Poverty_df.head()

In [116]:
Poverty_df["Pov_Group"] = Pov_Grp
Poverty_df["Pov_Pct"] = Pov_Pct
Poverty_df.to_csv("Poverty_dataframe.csv", index=False)

In [134]:
# Merge in the poverty statistics to the main dataframe
Poverty_df_ToMerge = Poverty_df[["Zipcode", "Household Income", "Per Capita Income", "Pvty Rate", "Unemployed Rate",
                                "Pov_Group", "Pov_Pct"]]
Poverty_df_ToMerge.head()
Final_SE_df = pd.merge(Final_SE_df, Poverty_df_ToMerge, on="Zipcode", how="left")
Final_SE_df.head()

,Zipcode,State,Urban Type,Center Point Lat,Center Point Lng,Population,Pop Density,Median Age,ZC Eth,Pct Dom Eth,...,Lng_Pct,House_Const_Yr,Med Gross Rent,Med Home_Val,Household Income,Per Capita Income,Pvty Rate,Unemployed Rate,Pov_Group,Pov_Pct
0,00601,PR,Mixed,18.180555,-66.749961,17982.0,279.449245,37.6,His,99.777555,...,87.932377,1982.0,359.0,104500.0,10816.0,7453.0,60.399288,12.306751,His,99.631710
1,00602,PR,Urbanized Area,18.361945,-67.175597,40260.0,1315.127560,39.5,His,93.594138,...,91.308992,1980.0,395.0,90200.0,16079.0,8474.0,53.949329,9.456036,His,93.687845
2,00603,PR,Urbanized Area,18.455183,-67.119887,52408.0,1657.641700,39.9,His,96.239124,...,89.867959,1977.0,384.0,128400.0,16804.0,10179.0,48.578461,7.615250,His,97.022664
3,00606,PR,Mixed,18.158345,-66.932911,6331.0,149.637193,40.8,His,99.873638,...,91.154636,1978.0,280.0,108500.0,12512.0,5863.0,64.286843,2.827357,His,99.803440
4,00610,PR,Urbanized Area,18.295366,-67.125135,28328.0,788.729257,40.2,His,98.838605,...,92.452697,1979.0,424.0,113200.0,17475.0,8452.0,49.438718,5.277464,His,98.821849


In [136]:
# This is the final Socioeconomics dataframe that we can use for analysis. There are other intricacies missing here, but
# we can always return to some of the intermediate dataframes to analyze those. 
# PLEASE SEE THE ACS Zipcode Socioeconomics - Testing sheet more more tests on this data
Final_SE_df.count()
Final_SE_df.to_csv("Final_SE_df.csv", index=False)